In [137]:
import matplotlib.pyplot as plt

import pandas as pd
import requests
import json
import csv

In [138]:


#Get and clean premier league data
bs_response = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')

bs_data = json.loads(bs_response.text)

#assign 
events = bs_data['events']
players = bs_data['elements']
teams = bs_data['teams'] 
positions = bs_data['element_types']

In [139]:
positions_list = []
for position in positions:
    positions_list.append([position['id'], position['singular_name']])

positions_df = pd.DataFrame(positions_list, columns=['element_type', 'position_name'])

positions_df

,element_type,position_name
0,1,Goalkeeper
1,2,Defender
2,3,Midfielder
3,4,Forward


In [140]:

#populate dictionary with ids and names of players from json
player_dict = {
    #Pierre-Emerick Aubameyang : 4,
    #Mohamed Salah: 254
}


#populate dictionary with ids and names of players from json
player_name = None
for player in players:
    p_id = player['id'] 
    player_name = player['first_name'] + ' ' + player['second_name'] 
    player_dict[p_id] =  player_name

player_id_df = pd.DataFrame.from_dict(player_dict, orient='index', columns=['name'])

player_id_df = player_id_df.reset_index() 
#print(player_id_df.head())


player_positions = []
for player in players:
    player_positions.append([player['id'], player['element_type']])

player_pos_df = pd.DataFrame(player_positions, columns=['player_id', 'element_type'])

player_pos_df.set_index('player_id')

player_pos_df = player_pos_df.merge(positions_df, left_on='element_type', right_on='element_type')

player_pos_df = player_pos_df.set_index('player_id')

player_pos_df
#reverse order of k,v to be able to perform name lookup
#inv_player_dict = { v: k for k,v in player_dict.items() }

,element_type,position_name
player_id,,
1,3,Midfielder
4,3,Midfielder
9,3,Midfielder
18,3,Midfielder
19,3,Midfielder
...,...,...
674,1,Goalkeeper
453,1,Goalkeeper
455,1,Goalkeeper


In [141]:


#create list of player ids and teams to join with stats

team_list = {}
for team in teams:
    team_list[team['id']] = team['name']

team_df = pd.DataFrame.from_dict(team_list, orient='index', columns=['team_name'])
#print(team_df.head()) # team_id: team_name

player_on_team = {}
for player in players:
    player_on_team[player['id']] = player['team']
player_on_team_df = pd.DataFrame.from_dict(player_on_team, orient='index', columns=['team_id'])

#join team list and player on team to add to csv file
players_teams_df = player_on_team_df.join(team_df, on='team_id')
players_teams_df = players_teams_df.reset_index()
players_teams_df.head(40)

,index,team_id,team_name
0,1,1,Arsenal
1,2,1,Arsenal
2,3,1,Arsenal
3,4,1,Arsenal
4,5,1,Arsenal
5,6,1,Arsenal
6,7,1,Arsenal
7,8,1,Arsenal
8,9,1,Arsenal
9,10,1,Arsenal


In [142]:
#Generate links for each player
# url = 'https://fantasy.premierleague.com/api/element-summary/' + str(player_dict[player]) + '/'
player_urls = []

for player in players:
    player_urls.append('https://fantasy.premierleague.com/api/element-summary/' + str(player['id']) + '/')
#appends urls to data frame
#Loop through player URLS and add the results to a dataframe and save it to a csv file.

In [143]:

players_df = None

# now we will open a file for writing
data_file = open('player_stats.csv', 'w')
 
# create the csv writer object
csv_writer = csv.writer(data_file)

count = 0
for url in player_urls:
    response = requests.get(url)
    player_data =  response.json() #creates python dictionary from json response
    
    for player in player_data['history']:
        if count == 0:
            header = player.keys()
            csv_writer.writerow(header)
            count+=1
        csv_writer.writerow(player.values())

data_file.close()

In [164]:
#data wrangling to combine team names and player names to csv file
#read csv file into pandas dataframe
df = pd.read_csv('player_stats.csv', header=0)



df_with_teams = df.merge(players_teams_df, left_on='element', right_on='index', how='inner')


df_with_teams = df_with_teams.merge(player_id_df, left_on='index', right_on='index', how='inner')
df_with_teams
df = df_with_teams



In [171]:
#adds gameweek as new column
gameweeks = set()
kickoff_time = df['kickoff_time']
for gameweek in kickoff_time:
    g = gameweek.split('T')[0]
    gameweeks.add(g)

gameweeks = list(gameweeks)
gameweeks = sorted(gameweeks, reverse=True)


new = df['kickoff_time'].str.split('T', n = 1, expand= True)

df['gameweek'] = new[0]



,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,...,value,transfers_balance,selected,transfers_in,transfers_out,index,team_id,team_name,name,gameweek
0,2,8,0,False,2020-09-12T11:30:00Z,0.0,3.0,1,0,0,...,70,0,76656,0,0,1,1,Arsenal,Mesut Özil,2020-09-12
1,9,19,0,True,2020-09-19T19:00:00Z,2.0,1.0,2,0,0,...,69,-16828,68335,995,17823,1,1,Arsenal,Mesut Özil,2020-09-19
2,23,11,0,False,2020-09-28T19:00:00Z,3.0,1.0,3,0,0,...,69,-11451,59793,675,12126,1,1,Arsenal,Mesut Özil,2020-09-28
3,29,15,0,True,2020-10-04T13:00:00Z,2.0,1.0,4,0,0,...,68,-5324,56403,647,5971,1,1,Arsenal,Mesut Özil,2020-10-04
4,44,12,0,False,2020-10-17T16:30:00Z,1.0,0.0,5,0,0,...,68,-4224,53689,616,4840,1,1,Arsenal,Mesut Özil,2020-10-17


,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,...,value,transfers_balance,selected,transfers_in,transfers_out,index,team_id,team_name,name,gameweek
player_id,,,,,,,,,,,,,,,,,,,,,
0,2,8,0,False,2020-09-12T11:30:00Z,0.0,3.0,1,0,0,...,70,0,76656,0,0,1,1,Arsenal,Mesut Özil,2020-09-12
1,9,19,0,True,2020-09-19T19:00:00Z,2.0,1.0,2,0,0,...,69,-16828,68335,995,17823,1,1,Arsenal,Mesut Özil,2020-09-19
2,23,11,0,False,2020-09-28T19:00:00Z,3.0,1.0,3,0,0,...,69,-11451,59793,675,12126,1,1,Arsenal,Mesut Özil,2020-09-28
3,29,15,0,True,2020-10-04T13:00:00Z,2.0,1.0,4,0,0,...,68,-5324,56403,647,5971,1,1,Arsenal,Mesut Özil,2020-10-04
4,44,12,0,False,2020-10-17T16:30:00Z,1.0,0.0,5,0,0,...,68,-4224,53689,616,4840,1,1,Arsenal,Mesut Özil,2020-10-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,89,20,0,True,2020-11-29T19:15:00Z,1.0,2.0,10,0,0,...,50,-15129,51099,682,15811,567,1,Arsenal,Thomas Partey,2020-11-29
996,106,17,1,False,2020-12-06T16:30:00Z,2.0,0.0,11,45,0,...,50,-10758,40546,525,11283,567,1,Arsenal,Thomas Partey,2020-12-06
997,109,4,0,True,2020-12-13T19:15:00Z,0.0,1.0,12,0,0,...,50,-5346,35876,702,6048,567,1,Arsenal,Thomas Partey,2020-12-13


## add position to csv

In [172]:
test = df.merge(player_pos_df, how='inner', left_on='element', right_on='player_id')
#df.to_csv('player_stats.csv')


test.to_csv('player_stats_w_pos.csv', index=True)
df = pd.read_csv('player_stats_w_pos.csv', header=0, index_col=0)


,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,team_id,team_name,name,element_type_x,position_name_x,element_type_y,position_name_y,gameweek,element_type,position_name
0,1,2,8,0,False,2020-09-12T11:30:00Z,0.0,3.0,1,0,...,1,Arsenal,Mesut Özil,3,Midfielder,3,Midfielder,2020-09-12,3,Midfielder
1,1,9,19,0,True,2020-09-19T19:00:00Z,2.0,1.0,2,0,...,1,Arsenal,Mesut Özil,3,Midfielder,3,Midfielder,2020-09-19,3,Midfielder
2,1,23,11,0,False,2020-09-28T19:00:00Z,3.0,1.0,3,0,...,1,Arsenal,Mesut Özil,3,Midfielder,3,Midfielder,2020-09-28,3,Midfielder
3,1,29,15,0,True,2020-10-04T13:00:00Z,2.0,1.0,4,0,...,1,Arsenal,Mesut Özil,3,Midfielder,3,Midfielder,2020-10-04,3,Midfielder
4,1,44,12,0,False,2020-10-17T16:30:00Z,1.0,0.0,5,0,...,1,Arsenal,Mesut Özil,3,Midfielder,3,Midfielder,2020-10-17,3,Midfielder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20140,649,259,2,0,False,2021-03-06T17:30:00Z,0.0,0.0,27,0,...,20,Wolves,Hugo Bueno,2,Defender,2,Defender,2021-03-06,2,Defender
20141,649,278,11,0,True,2021-03-15T20:00:00Z,0.0,1.0,28,0,...,20,Wolves,Hugo Bueno,2,Defender,2,Defender,2021-03-15,2,Defender
20142,649,298,19,0,True,2021-04-05T19:15:00Z,2.0,3.0,30,0,...,20,Wolves,Hugo Bueno,2,Defender,2,Defender,2021-04-05,2,Defender
20143,649,302,8,0,False,2021-04-09T19:00:00Z,0.0,1.0,31,0,...,20,Wolves,Hugo Bueno,2,Defender,2,Defender,2021-04-09,2,Defender
